In [5]:
# Due to the file system setting of the Jupyter Notebook, navigate to the package to use infra
%cd /home/jupyter/quant

/home/jupyter/quant


#### Create yfinance SQL tables for the first time

In [ ]:
%run -i 'notebooks/etl/yfinance/create_sql_tables.py'

In [1]:
from datetime import datetime

### Extract and insert stock price datato cloud storage and SQL

#### Batch Load Max history data

In [9]:
ticker = 'AAPL'
password = ''
table_name = 'quant_daily'
period = 'max'

In [10]:
# Recent executed on
print(datetime.now())
print("======================================================================================================================================================")

%run -i 'notebooks/etl/yfinance/load_add_info.py'
print("======================================================================================================================================================")
    

2020-03-15 22:00:42.626671
AAPL in year 1980 has been write to gs://zhongyuan-dw/equity/company/aapl/1980/history.csv
AAPL in year 1981 has been write to gs://zhongyuan-dw/equity/company/aapl/1981/history.csv
AAPL in year 1982 has been write to gs://zhongyuan-dw/equity/company/aapl/1982/history.csv
AAPL in year 1983 has been write to gs://zhongyuan-dw/equity/company/aapl/1983/history.csv
AAPL in year 1984 has been write to gs://zhongyuan-dw/equity/company/aapl/1984/history.csv
AAPL in year 1985 has been write to gs://zhongyuan-dw/equity/company/aapl/1985/history.csv
AAPL in year 1986 has been write to gs://zhongyuan-dw/equity/company/aapl/1986/history.csv
AAPL in year 1987 has been write to gs://zhongyuan-dw/equity/company/aapl/1987/history.csv
AAPL in year 1988 has been write to gs://zhongyuan-dw/equity/company/aapl/1988/history.csv
AAPL in year 1989 has been write to gs://zhongyuan-dw/equity/company/aapl/1989/history.csv
AAPL in year 1990 has been write to gs://zhongyuan-dw/equity/co

In [ ]:
# Insert data into SQL table

ts_pd.reset_index(inplace=True)

with connection.cursor() as cursor:
    for index, row in ts_pd.iterrows():
        print(row)
        sql = """
        INSERT INTO {table_name}([{ticker}], [{date}], [{open}], [{high}], [{low}], [{close}], [{volume}], [{year}], [{month}], [{day}], [{updated_at}]) 
        VALUES({ticker_v}, {date_v}, {open_v}, {high_v}, {low_v}, {close_v}, {volume_v}, {year_v}, {month_v}, {day_v}, {updated_at_v})
        """.format(table_name=table_name,
                   ticker=TICKER,
                   date=DATE,
                   open=OPEN, high=HIGH, low=LOW, close=CLOSE, volume=VOLUME,
                   year=YEAR, month=MONTH, day=DAY, updated_at=UPDATED_AT,
                   ticker_v=row[TICKER],
                   date_v=row[DATE],
                   open_v=row[OPEN], high_v=row[HIGH], low_v=row[LOW], close_v=row[CLOSE], volume_v=row[VOLUME],
                   year_v=row[YEAR], month_v=row[MONTH], day_v=row[DAY], updated_at_v=row[UPDATED_AT])

        cursor.execute(sql)

        # connection is not autocommit by default. So you must commit to save your changes.
        connection.commit()
